# SPC plot
Statistical control plot on PowderPro batch lab analysis data. The measured (numeric) end product parameters viscosity, assay, color, and MW (molecular weight) of the latest batches will be displayed.

## Initialization
Library imports and initialization of the client.

In [1]:
import os
import pandas as pd
import plotly.graph_objects as go
from trendminer.trendminer_client import TrendMinerClient  # Importing TrendMinerClient from trendminer package

# Retrieve the user token from the environment variables
token = os.environ["KERNEL_USER_TOKEN"]

# Create TrendMiner API object using the provided token
client = TrendMinerClient(token)  # Creating an instance of TrendMinerClient using the user token

## Loading the batch data
The PowderPro batch data can be loaded directly from a ContextHub view

In [2]:
# Load the ContextHub view named "PowderPro ALL" using the TrendMiner package
from trendminer.context.context_view import ContextView  # Import ContextView from the trendminer package
import uuid  # Import uuid module for creating UUIDs

# Create a ContextView object using the TrendMinerClient and the view ID
context_view = ContextView(client, "2c91808d6f8a87d3016ff30c578e0058")

# Load the view data into a DataFrame
df = context_view.load_view()

# Set the "start_date" column as the index and select specific columns for analysis
df = df.set_index("start_date").loc[:, ["ASSAY", "TMCOLOR", "TMMW", "TMVISCOSITY", "TMBATCHID", "TMGRADE"]]

# Rename the DataFrame columns for easier reference
df.columns = ["assay", "color", "mw", "viscosity", "batch", "grade"]

# Sort the DataFrame by the index (start_date)
df = df.sort_index()

## Plot generation
An interactive plot is generated using Plotly. The batch values are plotted, as well as the means and the upper and lower limits (taken as 3 standard deviations from the mean).

In [4]:
# Calculate the mean and standard deviation for each column
mean_values = df.mean()
std_values = df.std()

# Calculate upper and lower limits for the data points
upper_lim = mean_values + std_values * 3
lower_lim = mean_values - std_values * 3

# Initialize a Figure object
fig = go.Figure()


# List of variables
variables = ["assay", "color", "mw", "viscosity"]

# Colormap for different grades
colormap = {
    "Pharma": "#80c1ff",
    "Food": "#e6d1ae",
    "Industrial": "#b9b5b2",
}

            
fig.update_layout(
    #title="Viscosity",
    width=900,
    height=200,
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=False,
    plot_bgcolor="white",
)

# Add traces for each column in the DataFrame
for column in df.columns.to_list():
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[column],
            name=column,
            mode="markers",
            text=df['batch'],
            marker_color=df["grade"].map(colormap),
            visible=column=="viscosity",
        )
    )

# Add a line for the mean values
for variable, mean_value in zip(variables, mean_values):
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=[mean_value] * len(df),
            name=f'Mean',
            mode="lines",
            line=dict(color=colormap.get(variable, 'green')),
            visible=variable=="viscosity",
        )
    )
    
# Add lines for the upper limits
for variable, up_lim in zip(variables, upper_lim):
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=[up_lim] * len(df),
            name=f'up lim',
            mode="lines",
            line=dict(color=colormap.get(variable, 'red')),
            visible=variable=="viscosity",
        )
    )

# Add lines for the lower limits
for variable, low_lim in zip(variables, lower_lim):
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=[low_lim] * len(df),
            name=f'low lim',
            mode="lines",
            line=dict(color=colormap.get(variable, 'red')),
            visible=variable=="viscosity",
        )
    )

# Update the layout of the figure to include dropdown menu options
fig.update_layout(
    updatemenus=[
        go.layout.Updatemenu(
            active=0,
            buttons=list(
                [dict(
                    label='Viscosity',
                    method='update',
                    args=[
                        {'visible': [False, False, False, True, False, False, False, False, False, True, False, False, False, True, False, False, False, True]},
                        #{'title': 'Viscosity', 'showlegend': True}
                    ]),
                 dict(
                    label='Assay',
                    method='update',
                    args=[
                        {'visible': [True, False, False, False, False, False, True, False, False, False, True, False, False, False, True, False, False, False]},
                        #{'title': 'Assay', 'showlegend': True}
                    ]),
                 dict(
                    label='Color',
                    method='update',
                    args=[
                        {'visible': [False, True, False, False, False, False, False, True, False, False, False, True, False, False, False, True, False, False]},
                        #{'title': 'Color', 'showlegend': True}
                    ]),
                 dict(
                    label='mw',
                    method='update',
                    args=[
                        {'visible': [False, False, True, False, False, False, False, False, True, False, False, False, True, False, False, False, True, False]},
                        #{'title': 'mw', 'showlegend': True}
                    ]),
                 dict(
                    label='All',
                    method='update',
                    args=[
                        {'visible': [True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, True, True]},
                        #{'title': 'All', 'showlegend': True}
                    ])
                ])
            )
        ])
# Show the figure
fig.show()
